<a href="https://colab.research.google.com/github/Bhavya728/bhavya_machine_learning_UML501/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q.1

In [2]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MaxAbsScaler

cust_df = pd.read_csv('/content/AWCustomers.csv')
sales_df = pd.read_csv('/content/AWSales.csv')

combined_df = pd.merge(cust_df, sales_df, on='CustomerID')

combined_df['BirthDate'] = pd.to_datetime(combined_df['BirthDate'], errors='coerce')
combined_df['Age'] = datetime.now().year - combined_df['BirthDate'].dt.year

selected_cols = [
    'Gender',
    'MaritalStatus',
    'Age',
    'Education',
    'HomeOwnerFlag',
    'NumberCarsOwned',
    'NumberChildrenAtHome',
    'TotalChildren',
    'YearlyIncome',
    'BikeBuyer'
]

customer_df = combined_df[selected_cols]
print(customer_df.head())


  Gender MaritalStatus  Age        Education  HomeOwnerFlag  NumberCarsOwned  \
0      M             M   38        Bachelors              1                3   
1      M             M   53  Partial College              1                2   
2      F             S   40        Bachelors              0                3   
3      M             M   48  Partial College              1                2   
4      M             S   50  Partial College              1                1   

   NumberChildrenAtHome  TotalChildren  YearlyIncome  BikeBuyer  
0                     0              1         81916          1  
1                     1              2         81076          1  
2                     0              0         86387          1  
3                     1              2         61481          1  
4                     0              0         51804          1  


Q.2

In [4]:
## Handling missing values
print(customer_df.isnull().sum())
customer_df = customer_df.dropna()

## Normalization
scale_features = ['Age', 'YearlyIncome', 'TotalChildren']
abs_scaler = MaxAbsScaler()
scaled_output = abs_scaler.fit_transform(customer_df[scale_features])
customer_df[scale_features] = pd.DataFrame(scaled_output, columns=scale_features, index=customer_df.index)

## Discretization
customer_df['Age_Group'] = pd.cut(
    customer_df['Age'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Young', 'Middle-Aged', 'Senior']
)

## Standardization
from sklearn.preprocessing import StandardScaler
std_cols = ['NumberCarsOwned', 'NumberChildrenAtHome']
std_proc = StandardScaler()
customer_df[std_cols] = std_proc.fit_transform(customer_df[std_cols])

## Binarization
final_dataset = pd.get_dummies(
    customer_df,
    columns=['Gender', 'MaritalStatus', 'Education', 'Age_Group']
)

print(final_dataset.head())


Gender                  0
MaritalStatus           0
Age                     0
Education               0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
BikeBuyer               0
dtype: int64
        Age  HomeOwnerFlag  NumberCarsOwned  NumberChildrenAtHome  \
0  0.400000              1         1.892524             -0.594371   
1  0.557895              1         0.798389              1.163279   
2  0.421053              0         1.892524             -0.594371   
3  0.505263              1         0.798389              1.163279   
4  0.526316              1        -0.295746             -0.594371   

   TotalChildren  YearlyIncome  BikeBuyer  Gender_F  Gender_M  \
0       0.333333      0.588837          1     False      True   
1       0.666667      0.582798          1     False      True   
2       0.000000      0.620975          1      True     False   
3       0.666667      0.441944          1     False 

Q.3

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
rec1 = final_dataset.iloc[0]
rec2 = final_dataset.iloc[1]
vec1 = rec1.values
vec2 = rec2.values

## Simple Matching Coefficient
equal_count = np.sum(vec1 == vec2)
total_len = len(vec1)
smc_score = equal_count / total_len
print(smc_score)

## Jaccard Coefficient
common_ones = np.sum((vec1 == 1) & (vec2 == 1))
union_ones = np.sum((vec1 == 1) | (vec2 == 1))
jaccard_score = common_ones / union_ones
print(jaccard_score)

## Cosine
cosine_score = cosine_similarity([vec1], [vec2])
print(cosine_score)


0.631578947368421
0.7142857142857143
[[0.67485062]]
